In [ ]:
import time
import logging
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException

In [ ]:
# configure logging
logging.basicConfig(filename='error.log', level=logging.ERROR)

chrome_path = 'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
df = pd.read_excel(r'C:\Users\U6060174\OneDrive - Clarivate Analytics\Desktop\Files\NGB Yazaki\US\url_rerun_1.xlsx')

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = chrome_path

driver = webdriver.Chrome(options=chrome_options)
retry_count = 3  # number of times to retry if an error occurs
for i, row in df.iterrows():
    url = row['{URL}']
    name = row['{Name}']
    for _ in range(retry_count):  # retry loop
        try:
            driver.delete_all_cookies()
            driver.set_page_load_timeout(30)  # set a page load timeout of 30 seconds
            driver.get(url)
            time.sleep(25)
            html = driver.page_source
            with open(f'C:\\Users\\U6060174\\OneDrive - Clarivate Analytics\\Desktop\Files\\NGB Yazaki\\US\\Patent Center Page\\{name}.html', 'w', encoding='utf-8') as f:
                f.write(html)
                time.sleep(5)  # add a 5 seconds delay
            break  # break out of the retry loop if successful
        except (TimeoutException, WebDriverException) as e:
            logging.error(f"Error occurred while saving the page for {url}: {e}")
            print(f"Error occurred while saving the page for {url}: {e}")
            time.sleep(10)  # add a 10 seconds delay before retrying
    else:  # executed if the retry loop completes without success
        logging.error(f"Failed to save the page after {retry_count} retries for {url}")
        print(f"Failed to save the page after {retry_count} retries for {url}")
    
driver.quit()


In [ ]:
df_data = pd.DataFrame(columns=['Name', 'Data', 'Status'])

for i, row in df.iterrows():
    name = row['{Name}']
    file_path = f'C:\\Users\\U6060174\\OneDrive - Clarivate Analytics\\Desktop\\Files\\NGB Yazaki\\US\\Patent Center Page\\{name}.html'
    status = 'Success' # set default status as success
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            html = f.read()

        soup = BeautifulSoup(html, 'html.parser')

        text = soup.get_text(separator='\n')
        text = text.replace('<div>', '<div> ')
        text = text.replace('\n', ' \n')
    except Exception as e:
        print(f"Error occurred while processing the file for {name}: {e}")
        text = ''
        status = 'Error' # set status as error if there's an exception
    
    df_data = pd.concat([df_data, pd.DataFrame({'Name': name, 'Data': text, 'Status': status}, index=[0])])

df_data.to_excel(r'C:\Users\U6060174\OneDrive - Clarivate Analytics\Desktop\Files\NGB Yazaki\US\scraped_data.xlsx', index=False)
